In [18]:
import os
import pandas as pd
from eodhd import APIClient
from datetime import datetime, timedelta
import requests
import datetime
import pytz
import glob


In [16]:
api_key = os.getenv('EODHD_API_KEY')

stocks = [
    "AAPL",   # Apple Inc.
    "MSFT",   # Microsoft Corporation
    "AMZN",   # Amazon.com, Inc.
    "GOOGL",  # Alphabet Inc.
    "TSLA",   # Tesla, Inc.
    "NVDA",   # NVIDIA Corporation
    "META",   # Meta Platforms, Inc.
    "NFLX",   # Netflix, Inc.
    "AMD",    # Advanced Micro Devices, Inc.
    "INTC",   # Intel Corporation
    "BAC",    # Bank of America
    "JPM",    # JPMorgan Chase & Co.
    "WFC",    # Wells Fargo & Company
    "XOM",    # ExxonMobil
    "CVX",    # Chevron Corporation
    "PFE",    # Pfizer Inc.
    "MRK",    # Merck & Co.
    "UNH",    # UnitedHealth Group
    "V",      # Visa Inc.
    "MA"      # Mastercard Incorporated
]

start_date = "2025-04-01"
end_date = "2025-06-30"

In [12]:

def fetch_intraday_stock_data(stocks, start_date, end_date, api_key, output_dir="Intraday stocks"):
    """
    Fetches intraday (1-minute interval) intraday stock data for each of the specified stocks 
    over a multi-day period using default midnight boundaries.
    
    Instead of using specific trading hours, this function uses midnight ET as both the start 
    and the end reference. The period extends from midnight on start_date ET to midnight 
    on the day following end_date ET (thus covering the full end_date).
    
    Parameters:
      stocks : list or str
          One or more stock tickers (without the ".US" suffix), e.g., ['GOOGL', 'AAPL'].
      start_date : str
          Start date in "YYYY-MM-DD" format.
      end_date : str
          End date in "YYYY-MM-DD" format.
      api_key : str
          Your API key for the EODHD intraday API.
      output_dir : str, optional
          Directory in which to save CSV files. If None, files are saved in the current directory.
    
    Returns:
      results : dict
          A dictionary mapping each stock symbol to its DataFrame.
    """
    # Make sure stocks is a list.
    if isinstance(stocks, str):
        stocks = [stocks]
    
    # Define Eastern Time.
    et = pytz.timezone("US/Eastern")
    
    # Parse input dates.
    start_dt = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_dt   = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    
    # Define the period using midnight as the default:
    # Start at midnight on the start_date.
    start_et = et.localize(datetime.datetime(start_dt.year, start_dt.month, start_dt.day, 0, 0, 0))
    # End at midnight on the day after end_date.
    end_et = et.localize(datetime.datetime(end_dt.year, end_dt.month, end_dt.day, 0, 0, 0)) + datetime.timedelta(days=1)
    
    # Convert the Eastern Time values to UTC.
    start_utc = start_et.astimezone(pytz.utc)
    end_utc   = end_et.astimezone(pytz.utc)
    
    # Convert to Unix timestamps.
    from_timestamp = int(start_utc.timestamp())
    to_timestamp   = int(end_utc.timestamp())
    
    print("Requesting data from (UTC):", start_utc.isoformat())
    print("Until (UTC):", end_utc.isoformat())
    
    results = {}
    
    for stock in stocks:
        # For US tickers, the API expects a suffix ".US".
        ticker = f"{stock}.US"
        url = (
            f"https://eodhd.com/api/intraday/{ticker}"
            f"?interval=1m&api_token={api_key}&fmt=json"
            f"&from={from_timestamp}&to={to_timestamp}"
        )
        
        print(f"\nFetching data for {stock} from URL:")
        print(url)
        
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error fetching data for {stock}: {response.status_code}")
            print(response.text)
            continue
        
        data = response.json()
        df = pd.DataFrame(data)
        
        # Construct a file name that includes the stock symbol and the date range.
        file_name = f"{stock}_{start_dt.strftime('%Y%m%d')}_{end_dt.strftime('%Y%m%d')}.csv"
        if output_dir:
            file_path = os.path.join(output_dir, file_name)
        else:
            file_path = file_name
        
        df.to_csv(file_path, index=False)
        print(f"Saved data for {stock} to {file_path}")
        
        results[stock] = df
    
    return results




In [17]:
data = fetch_intraday_stock_data(stocks, start_date, end_date, api_key)

Requesting data from (UTC): 2025-04-01T04:00:00+00:00
Until (UTC): 2025-07-01T04:00:00+00:00

Fetching data for AAPL from URL:
https://eodhd.com/api/intraday/AAPL.US?interval=1m&api_token=67b5ff99b63306.32749083&fmt=json&from=1743480000&to=1751342400
Saved data for AAPL to Intraday stocks\AAPL_20250401_20250630.csv

Fetching data for MSFT from URL:
https://eodhd.com/api/intraday/MSFT.US?interval=1m&api_token=67b5ff99b63306.32749083&fmt=json&from=1743480000&to=1751342400
Saved data for MSFT to Intraday stocks\MSFT_20250401_20250630.csv

Fetching data for AMZN from URL:
https://eodhd.com/api/intraday/AMZN.US?interval=1m&api_token=67b5ff99b63306.32749083&fmt=json&from=1743480000&to=1751342400
Saved data for AMZN to Intraday stocks\AMZN_20250401_20250630.csv

Fetching data for GOOGL from URL:
https://eodhd.com/api/intraday/GOOGL.US?interval=1m&api_token=67b5ff99b63306.32749083&fmt=json&from=1743480000&to=1751342400
Saved data for GOOGL to Intraday stocks\GOOGL_20250401_20250630.csv

Fetchi

In [10]:
data['GOOGL']

,timestamp,gmtoffset,datetime,open,high,low,close,volume
0,1738573200,0,2025-02-03 09:00:00,200.01000000,200.09000000,198.26000000,200.03000000,5135
1,1738573260,0,2025-02-03 09:01:00,200.08000000,200.09000000,199.86000000,199.90000000,2165
2,1738573320,0,2025-02-03 09:02:00,199.91000000,200.00000000,199.46000000,199.88000000,4996
3,1738573380,0,2025-02-03 09:03:00,199.77000000,200.09000000,199.75000000,199.84000000,5094
4,1738573440,0,2025-02-03 09:04:00,199.90000000,200.01000000,199.80000000,200.00000000,2940
...,...,...,...,...,...,...,...,...
68842,1747353300,0,2025-05-15 23:55:00,164.72430000,164.72430000,164.68000000,164.68000000,25
68843,1747353360,0,2025-05-15 23:56:00,164.68000000,164.68000000,164.52050000,164.67000000,218
68844,1747353420,0,2025-05-15 23:57:00,164.60000000,164.73000000,164.60000000,164.68000000,49
68845,1747353480,0,2025-05-15 23:58:00,164.67000000,164.72880000,164.55010000,164.60000000,136


In [19]:
def merge_stock_files(stock, file_dir, date_col="datetime", output_dir="Intraday stocks"):
    """
    Merge multiple CSV files for a given stock by reading, concatenating, 
    and sorting them in chronological order. The final output file is named using 
    the earliest date (from the first file) and the latest date (from the last file).
    
    Parameters:
      stock    : str
          The stock ticker (e.g., "META").
      file_dir : str
          The directory where the CSV files are stored.
      date_col : str, default "Date"
          The name of the column containing the datetime.
          (This column will be parsed as a datetime and set as the index.)
      output_dir : str, optional
          Directory where the merged CSV file should be saved. 
          If None, the file is saved in the current directory.
    
    Returns:
      merged_df : pd.DataFrame
          A DataFrame containing the merged data in chronological order.
      final_file_path : str
          The path to the saved CSV file.
    """
    # Create a pattern to match files starting with the stock ticker (e.g., "META_*.csv")
    pattern = os.path.join(file_dir, f"{stock}_*.csv")
    file_list = sorted(glob.glob(pattern))
    
    if not file_list:
        raise ValueError(f"No files found for ticker {stock} in directory: {file_dir}")
    
    df_list = []
    for file in file_list:
        df = pd.read_csv(file, parse_dates=[date_col])
        df.set_index(date_col, inplace=True)
        df_list.append(df)
    
    # Concatenate and sort the resultant DataFrame.
    merged_df = pd.concat(df_list)
    merged_df.sort_index(inplace=True)
    
    # Retrieve the earliest and latest dates encountered.
    earliest_date = merged_df.index.min()
    latest_date = merged_df.index.max()
    
    earliest_str = earliest_date.strftime("%Y%m%d")
    latest_str = latest_date.strftime("%Y%m%d")
    
    # Construct the final file name using the stock ticker and these dates.
    final_filename = f"{stock}_{earliest_str}_{latest_str}.csv"
    if output_dir:
        file_path = os.path.join(output_dir, final_filename)
    else:
        file_path = final_filename
    
    # Save the merged DataFrame.
    merged_df.to_csv(file_path)
    print(f"Merged file saved as: {file_path}")
    
    return merged_df, file_path



In [22]:

# Dictionary to store merged DataFrames for each stock.
merged_data = {}

for stock in stocks:
    try:
        merged_df, final_path = merge_stock_files(
            stock,
            file_dir="Intraday stocks",
            date_col="datetime",  # Adjust this if your date column is called something else
            output_dir="Intraday stocks"
        )
        merged_data[stock] = merged_df
        print(f"Merged data for {stock} saved to {final_path}")
    except ValueError as e:
        print(e)

# Now merged_data contains a DataFrame for each stock, merged in the correct chronological order.


Merged file saved as: Intraday stocks\AAPL_20250102_20250603.csv
Merged data for AAPL saved to Intraday stocks\AAPL_20250102_20250603.csv
Merged file saved as: Intraday stocks\MSFT_20250102_20250603.csv
Merged data for MSFT saved to Intraday stocks\MSFT_20250102_20250603.csv
Merged file saved as: Intraday stocks\AMZN_20250102_20250603.csv
Merged data for AMZN saved to Intraday stocks\AMZN_20250102_20250603.csv
Merged file saved as: Intraday stocks\GOOGL_20250102_20250603.csv
Merged data for GOOGL saved to Intraday stocks\GOOGL_20250102_20250603.csv
Merged file saved as: Intraday stocks\TSLA_20250102_20250603.csv
Merged data for TSLA saved to Intraday stocks\TSLA_20250102_20250603.csv
Merged file saved as: Intraday stocks\NVDA_20250102_20250603.csv
Merged data for NVDA saved to Intraday stocks\NVDA_20250102_20250603.csv
Merged file saved as: Intraday stocks\META_20250102_20250603.csv
Merged data for META saved to Intraday stocks\META_20250102_20250603.csv
Merged file saved as: Intraday 